In [1]:
import pandas as pd
import kinproc as kp
from kinproc.jtsFunctions import *
from Kinematics_Dataframe import Kinematics_Dataframe
from utils import *
from Meta_Algorithm import *
import pickle
import numpy as np
import sklearn

In [2]:
def load_sklearn_model_from_pickle(model_name):
    import pickle

    with open(model_name, "rb") as file:
        model = pickle.load(file)
    return model

In [3]:
svm_model = load_sklearn_model_from_pickle("svm_model.sav")
stacked_model = load_sklearn_model_from_pickle("stacked_model.sav")
knn_model = load_sklearn_model_from_pickle("knn_model.sav")

# Creating dataframes
Loading the dataframes for the totality of the data as well as the specific test set that we care about.

Because the full df has more informationm, we we are going to grab specific image sequences from the test data and append the same sequences from the total dataframe.

In [4]:
dataframe_name = "./all_kinematics_dataframe.pkl"
anon_dataframe_path = "./ANONYMIZED_DATA.kin"
# load dataframe

df = pd.read_pickle(dataframe_name)
test_data_df = pd.read_pickle(anon_dataframe_path)

In [5]:
Study_1_df = Symmetry_Trap_Solver(test_data_df, "Study_1")
Study_3_df = Symmetry_Trap_Solver(test_data_df, "Study_5")
Study_6_df = Symmetry_Trap_Solver(test_data_df, "Study_6")
Study_7_df = Symmetry_Trap_Solver(test_data_df, "Study_7")

In [6]:
pat, ses, mvt = Study_1_df.unique_pat_sess_mvt[0]
dict_key = pat + "_" + ses + "_" + mvt
# grab the data for the first patient, session, and movement

Study_1_all = df[
    (df["Patient"] == pat) & (df["Session"] == ses) & (df["Movement"] == mvt)
].reset_index(drop=True)

# add side information to testing dataframe
side_dict = {}

##for subdf in Study_1_df.sub_dataframes:
 #   print(subdf.split("_"))
 #   pat, _, sess, mvt, mvt_num = subdf.split("_")
 #   sess = "Session_" + sess
 ###   mvt = mvt + "_" + mvt_num
    ## print(pat,sess,mvt)
    #side = df[
    #    (df["Patient"] == pat) & (df["Session"] == sess) & (df["Movement"] == mvt)
    #]["Side"].values[0]
    # print(side)

for study_df in [Study_1_df, Study_3_df, Study_6_df, Study_7_df]:
    for unique_psm in study_df.unique_pat_sess_mvt:
        pat, ses, mvt = unique_psm
        dict_key = pat + "_" + ses + "_" + mvt
        #get study number
        print("study number: ", study_df.study, dict_key)
        side = df[
            (df["Patient"] == pat) & (df["Session"] == ses) & (df["Movement"] == mvt)
        ]["Side"].values[0]
        # print(side)
        side_dict[dict_key] = side
        # calculate the symmetric pose for each study

study number:  Study_1 Patient_1_Session_1_Movement_1
study number:  Study_1 Patient_2_Session_1_Movement_1
study number:  Study_1 Patient_2_Session_1_Movement_2
study number:  Study_1 Patient_3_Session_1_Movement_2
study number:  Study_1 Patient_4_Session_1_Movement_2
study number:  Study_1 Patient_5_Session_1_Movement_2
study number:  Study_1 Patient_6_Session_1_Movement_2
study number:  Study_5 Patient_1_Session_1_Movement_1
study number:  Study_5 Patient_1_Session_1_Movement_2
study number:  Study_5 Patient_1_Session_1_Movement_3
study number:  Study_5 Patient_1_Session_1_Movement_4
study number:  Study_6 Patient_3_Session_1_Movement_1
study number:  Study_6 Patient_3_Session_1_Movement_2
study number:  Study_6 Patient_3_Session_1_Movement_3
study number:  Study_6 Patient_3_Session_1_Stair_1


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
side_dict

In [ ]:
pat_sess_mvt = Study_1_df.unique_pat_sess_mvt[0]
print(pat_sess_mvt)
pat_sess_mvt_key = pat_sess_mvt[0] + "_" + pat_sess_mvt[1] + "_" + pat_sess_mvt[2]
Study_1_df.sub_dataframes[pat_sess_mvt_key].calculate_symmetric_pose(
    side_dict, plot=False
)

def mvt_key_creator(dataframe, idx):
    pat_sess_mvt = dataframe.unique_pat_sess_mvt[idx]
    pat_sess_mvt_key = (
        pat_sess_mvt[0] + "_" + pat_sess_mvt[1] + "_" + pat_sess_mvt[2]
    )
    return pat_sess_mvt_key

# Applying Machine Learning Models to the dataset

In [ ]:
Study_1_df.sub_dataframes[pat_sess_mvt_key].correct_kinematics_with_machine_learning(
    svm_model
)
Study_1_df.sub_dataframes[pat_sess_mvt_key].create_spline()

In [ ]:
Xtest = np.load("X.npy")
ttest = np.load("t.npy")


Study_1_df.sub_dataframes[pat_sess_mvt_key].correct_kinematics_with_machine_learning(
    svm_model
)
Study_1_df.sub_dataframes[pat_sess_mvt_key].create_spline()
Study_1_df.sub_dataframes[pat_sess_mvt_key].apply_spline()
Study_1_df.sub_dataframes[pat_sess_mvt_key].confusion_matrix()
Study_1_df.sub_dataframes[pat_sess_mvt_key].plotting_all_kinematics()

In [ ]:
def ml_workflow(model):
    total_tp = 0
    total_tn = 0
    total_fp = 0
    total_fn = 0
    sym_dist_correct = []
    sym_dist_incorrect = []
    
    for study_df in [Study_1_df, Study_3_df, Study_6_df, Study_7_df]:
        for psm_key in study_df.sub_dataframes.keys():
            print(psm_key)
            study_df.sub_dataframes[psm_key].calculate_symmetric_pose(side_dict)
            study_df.sub_dataframes[psm_key].correct_kinematics_with_machine_learning(
                model
            )
            study_df.sub_dataframes[psm_key].create_spline()
            study_df.sub_dataframes[psm_key].apply_spline()
            tp, tn, fp, fn = study_df.sub_dataframes[psm_key].confusion_matrix()
            total_tp += tp
            total_tn += tn
            total_fp += fp
            total_fn += fn
            
            sdc, sdi = study_df.sub_dataframes[psm_key].determine_symmetry_distance_for_each_frame()
            sym_dist_correct.append(sdc)
            sym_dist_incorrect.append(sdi)
    return total_tp, total_tn, total_fp, total_fn, sym_dist_correct, sym_dist_incorrect
    

In [ ]:
tp,tn,fp,fn, sym_dist_corr, sym_dist_incorr = ml_workflow(stacked_model)

In [ ]:
def plot_confusion_matrix(tp,tn,fp,fn):
    print(np.array([[tp,fp],[fn,tn]]))

In [ ]:
plot_confusion_matrix(tp,tn,fp,fn)

In [ ]:
def classification_metrics(tp,tn,fp,fn, print_vals = True):
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    recall = tp/(tp+fn)
    f1 = 2*precision*recall/(precision+recall)
    if print_vals:
        print("Accuracy: ", np.round(accuracy,3))
        print("Precision: ", np.round(precision,3))
        print("Recall: ", np.round(recall,3))
        print("F1: ", np.round(f1,3))
        print("Sensitivity: ", np.round(sensitivity,3))
        print("Specificity: ", np.round(specificity,3))
    return accuracy, precision, recall, f1, sensitivity, specificity

In [ ]:
acc, prec, rec, f1, sens, spec = classification_metrics(tp,tn,fp,fn)

In [ ]:
# get average of all values in sym dist corr 
# list of numpy arrays of different lengths
# average them all together

corr_sym_dist = np.array([item for sublist in sym_dist_corr for item in sublist])
print(corr_sym_dist.shape)
incorr_sym_dist = np.array([item for sublist in sym_dist_incorr for item in sublist])
print(incorr_sym_dist.shape)
print("Correct Frames symmetry distance: ", np.mean(corr_sym_dist))
print("Incorrect Frames symmetry distance: ",np.mean(incorr_sym_dist))

In [ ]:
# Create histogram of symmetry distances
# normalize histograms
# plot on same plot

import matplotlib.pyplot as plt
# plot with normalized values
plt.hist(corr_sym_dist, bins=20, density=True, alpha=0.5)
plt.hist(incorr_sym_dist, bins=10, density=True, alpha=0.5)
plt.show()

plt.hist(corr_sym_dist, bins=20, alpha=0.5)
plt.hist(incorr_sym_dist, bins=10, alpha=0.5)
plt.show()

In [ ]:
Study_1_df.sub_dataframes[pat_sess_mvt_key].create_kinematics_with_symmetry_traps(plot=True)

In [ ]:
Study_6_df.sub_dataframes[mvt_key_creator(Study_6_df, 12)].create_kinematics_with_symmetry_traps(plot=True)
#Study_3_df.sub_dataframes[mvt_key_creator(Study_3_df, 0)].create_kinematics_with_symmetry_traps(plot=True)
